imports

In [8]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Définition des profils types de personnes


In [9]:
profiles = [
    {"name": "Personne Matinale", "arrival": (7, 9), "departure": (15, 17), "temp_pref": (20, 22)},
    {"name": "Personne Standard", "arrival": (8, 10), "departure": (17, 19), "temp_pref": (21, 23)},
    {"name": "Personne Tardive", "arrival": (10, 12), "departure": (19, 21), "temp_pref": (22, 24)},
    {"name": "Aléatoire", "arrival": (7, 12), "departure": (15, 21), "temp_pref": (20, 24)},
]

Génération des utilisateurs selon les profils

In [12]:
# Fixer la graine aléatoire pour reproductibilité
random.seed(42)
np.random.seed(42)

# Liste de prénoms pour générer des utilisateurs uniques
names = ["Alice", "Bob", "Charlie", "David", "Emma", "Fiona", "George", "Hannah", "Ian", "Julia", "Kevin", "Liam"]

# Génération aléatoire du nombre de personnes par profil
num_users_per_profile = {p["name"]: random.randint(3, 6) for p in profiles}

# Générer des noms uniques pour chaque utilisateur
users = []
for profile, count in num_users_per_profile.items():
    for i in range(count):
        users.append({"user_id": f"{random.choice(names)}_{i}", "profile": profile})

In [17]:
# Génération des données pour une période donnée
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 3, 1)
time_step = timedelta(minutes=10)  # Une donnée toutes les 10 minutes

On peut : 
- Appliquer des profils sur les ouvertures/fermeture de fenetre
- Voir si je garde des noms ou si je donne les noms des bureaux 

Prompts données à ChatGPT qui montrent ce que j'ai voulu rajouter (et que je n'ai pas encore rajouté)
```
J'aimerai simuler un nombre aléatoire de personnes de chaque profile, et que le nombre de personne simulées puisse être toujours le même si souhaité (comme random_state=42).
J'aimerai aussi des noms unique à chaque utilisateur. 

Ensuite, j'aimerai utiliser un algorithme type classification ou clustering qui permet de retrouver le profil des utilisateurs en fonction de leur données. 

Comment également simuler des pauses longues (entre 1h et 2h) pour le repas et des pauses (pour le café ou autre) plus ou moins aléatoires et de durée plus courte (entre 5min et 20min) ? 

Aussi, j'aimerai simuler le fait que les utilisateurs peuvent modifier la température fixée pour le chaufffage et que la température intérieure varie en fonction de celle-ci.

J'aimerai simuler le fait que les bureaux ont des isolations variables, avec un indice d'isolation, moins cet indice est important, plus la température intérieure est influencée par la température extérieure.

Egalement, j'aimerai aussi simuler le fait que si la fenetre est ouverte, la température intérieure se rapproche plus rapidement de la température extérieure. ```

In [18]:
# Initialisation du dataset
data = []

# Simulation sur la période définie
current_time = start_date
while current_time < end_date:
    for user in users:
        profile = next(p for p in profiles if p["name"] == user["profile"])
        arrival_hour = random.randint(*profile["arrival"])
        departure_hour = random.randint(*profile["departure"])
        temp_pref = random.uniform(*profile["temp_pref"])

        # Création des horaires de présence
        arrival_time = current_time.replace(hour=arrival_hour, minute=0)
        departure_time = current_time.replace(hour=departure_hour, minute=0)

        # Simuler la détection de présence (1 si la personne est présente, 0 sinon)
        presence = 1 if arrival_time <= current_time <= departure_time else 0

        # Température intérieure simulée (dépend de la présence et du réglage préféré)
        if presence:
            temp_int = temp_pref + random.uniform(-0.5, 0.5)  # Légère variation autour de la préférence
        else:
            temp_int = random.uniform(16, 20)  # Baisse de la température en absence

        # Température extérieure simulée (variation jour/nuit)
        hour = current_time.hour
        temp_ext = random.uniform(5, 15) if 6 <= hour <= 18 else random.uniform(-2, 7)

        # Fenêtre ouverte ? (10% de probabilité si présent, 5% sinon)
        window_open = 1 if (random.random() < (0.1 if presence else 0.05)) else 0

        # Chauffage/clim activé ? (si présence et température intérieure trop éloignée de la préférence)
        heating_on = 1 if presence and temp_int < temp_pref - 1 else 0
        cooling_on = 1 if presence and temp_int > temp_pref + 1 else 0

        # Enregistrement des données
        data.append([
            current_time, user["user_id"], presence, temp_int, temp_ext, window_open, heating_on, cooling_on
        ])

    current_time += time_step  # Avance dans le temps

# Création du DataFrame
df = pd.DataFrame(data, columns=["timestamp", "user_id", "presence", "temp_int", "temp_ext", "window_open", "heating_on", "cooling_on"])

# Sauvegarde au format CSV
df.to_csv("simulation_bureau.csv", index=False)
# Affichage des 10 premières lignes
print(df.head())


   timestamp    user_id  presence   temp_int  temp_ext  window_open  \
0 2024-02-01    David_0         0  18.047866  4.054686            0   
1 2024-02-01  Charlie_1         0  17.906469  2.588491            0   
2 2024-02-01     Liam_2         0  16.723637 -1.827051            0   
3 2024-02-01      Bob_0         0  18.990541 -0.309030            0   
4 2024-02-01    Kevin_1         0  18.658564  3.609939            0   

   heating_on  cooling_on  
0           0           0  
1           0           0  
2           0           0  
3           0           0  
4           0           0  


In [19]:
df.describe()

,timestamp,presence,temp_int,temp_ext,window_open,heating_on,cooling_on
count,62640,62640.000000,62640.000000,62640.000000,62640.000000,62640.0,62640.0
mean,2024-02-15 11:55:00,0.367593,19.529945,6.546132,0.069045,0.0,0.0
min,2024-02-01 00:00:00,0.000000,16.000045,-1.999604,0.000000,0.0,0.0
25%,2024-02-08 05:57:30,0.000000,17.572305,2.923009,0.000000,0.0,0.0
50%,2024-02-15 11:55:00,0.000000,19.164986,6.344954,0.000000,0.0,0.0
75%,2024-02-22 17:52:30,1.000000,21.638138,10.383077,0.000000,0.0,0.0
max,2024-02-29 23:50:00,1.000000,24.493157,14.999719,1.000000,0.0,0.0
std,NaN,0.482153,2.309617,4.641013,0.253533,0.0,0.0
